In [1]:
import pandas as pd 
code_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13', header=0)[0] 

# 종목코드가 6자리이기 때문에 6자리를 맞춰주기 위해 설정해줌 
code_df.종목코드 = code_df.종목코드.map('{:06d}'.format) 

# 우리가 필요한 것은 회사명과 종목코드이기 때문에 필요없는 column들은 제외해준다. 
code_df = code_df[['회사명', '종목코드']] 

# 한글로된 컬럼명을 영어로 바꿔준다. 
code_df = code_df.rename(columns={'회사명': 'name', '종목코드': 'code'}) 

code_df.head()


,name,code
0,DSR,155660
1,GS글로벌,001250
2,HSD엔진,082740
3,LG이노텍,011070
4,LS산전,010120


In [16]:
# 종목 이름을 입력하면 종목에 해당하는 코드를 불러와 
# 네이버 금융(http://finance.naver.com)에 넣어줌 
def get_url(item_name, code_df): 
    code = code_df.query("name=='{}'".format(item_name))['code'].to_string(index=False).strip()
    url = 'http://finance.naver.com/item/sise_day.nhn?code={code}'.format(code=code) 
    print("요청 URL = {}".format(url)) 
    return url 

# 신라젠의 일자데이터 url 가져오기 
item_name='대한항공' 
url = get_url(item_name, code_df) 
# 일자 데이터를 담을 df라는 DataFrame 정의 
df = pd.DataFrame() 
# 1페이지에서 20페이지의 데이터만 가져오기 
for page in range(1, 61): 
    pg_url = '{url}&page={page}'.format(url=url, page=page) 
    df = df.append(pd.read_html(pg_url, header=0)[0], ignore_index=True) 
    
# df.dropna()를 이용해 결측값 있는 행 제거 
df = df.dropna() 
    
# 상위 5개 데이터 확인하기 
df.head()


요청 URL = http://finance.naver.com/item/sise_day.nhn?code=003490


,날짜,종가,전일비,시가,고가,저가,거래량
1,2020.02.20,23650.0,550.0,24450.0,24700.0,23650.0,954991.0
2,2020.02.19,24200.0,250.0,24850.0,24850.0,24000.0,681276.0
3,2020.02.18,24450.0,850.0,25250.0,25250.0,24400.0,891527.0
4,2020.02.17,25300.0,350.0,25550.0,25650.0,25100.0,462173.0
5,2020.02.14,25650.0,400.0,26050.0,26150.0,25600.0,425300.0


In [17]:
# 한글로 된 컬럼명을 영어로 바꿔줌 
df = df.rename(columns= {'날짜': 'date', '종가': 'close', '전일비': 'diff', '시가': 'open', '고가': 'high', '저가': 'low', '거래량': 'volume'})

# 데이터의 타입을 int형으로 바꿔줌 
df[['close', 'diff', 'open', 'high', 'low', 'volume']] \
    = df[['close', 'diff', 'open', 'high', 'low', 'volume']].astype(int) 

# 컬럼명 'date'의 타입을 date로 바꿔줌 
df['date'] = pd.to_datetime(df['date']) 
# 일자(date)를 기준으로 오름차순 정렬 
df = df.sort_values(by=['date'], ascending=True) 
# 상위 5개 데이터 확인 
df.head()

,date,close,diff,open,high,low,volume
898,2017-09-05,32150,200,32050,32250,31750,370852
897,2017-09-06,31850,300,32200,32250,31700,382256
896,2017-09-07,31750,100,31950,32200,31700,460676
895,2017-09-08,31750,0,31800,31950,31250,375230
894,2017-09-11,32750,1000,32100,33100,31900,568569


In [18]:
def get_macd(df, short=12, long=26, t=9): 
    # 입력받은 값이 dataframe이라는 것을 정의해줌 
    df = pd.DataFrame(df) 
    # MACD 관련 수식 
    ma_12 = df.close.ewm(span=12).mean() # 단기(12) EMA(지수이동평균) 
    ma_26 = df.close.ewm(span=26).mean() # 장기(26) EMA 
    macd = ma_12 - ma_26 # MACD 
    macds = macd.ewm(span=9).mean() # Signal 
    macdo = macd - macds # Oscillator 
    df = df.assign(macd=macd, macds=macds, macdo=macdo).dropna() 
    return df

df = get_macd(df) 
# 상위 5개 데이터 확인 
df.head()


,date,close,diff,open,high,low,volume,macd,macds,macdo
898,2017-09-05,32150,200,32050,32250,31750,370852,0.000000,0.000000,0.000000
897,2017-09-06,31850,300,32200,32250,31700,382256,-6.730769,-3.739316,-2.991453
896,2017-09-07,31750,100,31950,32200,31700,460676,-11.677330,-6.992601,-4.684729
895,2017-09-08,31750,0,31800,31950,31250,375230,-13.640941,-9.244748,-4.396193
894,2017-09-11,32750,1000,32100,33100,31900,568569,25.371540,1.052815,24.318725


In [21]:
from plotly import subplots 
import plotly.offline as offline 
import plotly.graph_objs as go 
macd = go.Scatter( x=df.date, y=df['macd'], name="MACD") 
signal = go.Scatter( x=df.date, y=df['macds'], name="Signal") 
oscillator = go.Bar( x=df.date, y=df['macdo'], name="oscillator") 
trade_volume = go.Bar( x=df.date, y=df['volume'], name="volume") 
data = [macd, signal, oscillator] 

# data = [celltrion] 
layout = go.Layout(title='{} MACD 그래프'.format(item_name)) 
fig = subplots.make_subplots(rows=2, cols=1, shared_xaxes=True) 
for trace in data: 
    fig.append_trace(trace, 1,1) 
fig.append_trace(trade_volume, 2,1) 
# fig = go.Figure(data=data, layout=layout) 

offline.iplot(fig)